In [14]:
import networkx as nx
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
from shapely.wkt import loads

# 文件路径
nodelist_file = 'nodelist_upd.csv'
edgelist_file = 'edgelist_upd.csv'
path_file = 'path_upd.csv'
poi_file = 'poi_upd.csv'

# 创建图
G = nx.MultiDiGraph()

# 读取并添加节点 (nodelist 和 poi_list)
nodelist = pd.read_csv(nodelist_file)
for index, row in nodelist.iterrows():
    geom = loads(row['geometry'])
    row_data = row.to_dict()
    row_data.pop('x', None)  # 移除 'x'
    row_data.pop('y', None)  # 移除 'y'
    G.add_node(row['id'], x=geom.x, y=geom.y, **row_data)

poi_list = pd.read_csv(poi_file)
for index, row in poi_list.iterrows():
    geom = loads(row['geometry'])
    row_data = row.to_dict()
    row_data.pop('x', None)  # 移除 'x'
    row_data.pop('y', None)  # 移除 'y'
    G.add_node(row['osmid'], x=geom.x, y=geom.y, poi=True, **row_data)

# 从path_upd.csv中添加交点作为新节点
path_data = pd.read_csv(path_file)
for index, row in path_data.iterrows():
    geom = loads(row['geometry'])
    # 假设交点在线段的末端
    G.add_node(f"cross_{index}", x=geom.coords[-1][0], y=geom.coords[-1][1], path=True)

# 读取并添加边 (edgelist 和 path_data)
edgelist = pd.read_csv(edgelist_file)
for index, row in edgelist.iterrows():
    G.add_edge(row['start_node'], row['end_node'], **row.to_dict())

for index, row in path_data.iterrows():
    # 连接路径的起点和交点
    G.add_edge(row['start_node'], f"cross_{index}", **row.to_dict())

# 节点转换为GeoDataFrame
node_data = []
for node, data in G.nodes(data=True):
    if 'x' in data and 'y' in data:
        geom = Point(data['x'], data['y'])
        node_data.append({**data, 'geometry': geom})
node_gdf = gpd.GeoDataFrame(node_data)

# 边转换为GeoDataFrame
edge_data = []
for start_node, end_node, data in G.edges(data=True):
    if 'geometry' in data and isinstance(data['geometry'], str):
        try:
            geom = loads(data['geometry'])
        except Exception:
            continue  # 如果无法解析几何数据，跳过该边
    else:
        if 'x' in G.nodes[start_node] and 'y' in G.nodes[start_node] and 'x' in G.nodes[end_node] and 'y' in G.nodes[end_node]:
            start_geom = Point(G.nodes[start_node]['x'], G.nodes[start_node]['y'])
            end_geom = Point(G.nodes[end_node]['x'], G.nodes[end_node]['y'])
            geom = LineString([start_geom, end_geom])
        else:
            continue  # 跳过没有完整坐标的边
    edge_data.append({**data, 'geometry': geom})
edge_gdf = gpd.GeoDataFrame(edge_data)

# 导出为GeoJSON
node_gdf.to_file('nodes.geojson', driver='GeoJSON')
edge_gdf.to_file('edges.geojson', driver='GeoJSON')


C:\Users\Ven\AppData\Local\Temp\ipykernel_7700\746793276.py:41: DtypeWarning: Columns (5,8,13) have mixed types. Specify dtype option on import or set low_memory=False.
  edgelist = pd.read_csv(edgelist_file)
